# Financial Fraud Detection: A Comprehensive Machine Learning Approach


## 1. Introduction

In this notebook, I embark on a detailed machine learning project focused on detecting fraudulent financial transactions. I'm utilizing a rich dataset that combines transaction records, customer information, and card data. My primary goal is to build a robust fraud detection model, addressing common challenges such as severe class imbalance and data quality issues.

Fraud detection is a critical application of machine learning in the financial sector. It presents unique complexities:
-   **Class Imbalance:** Fraudulent transactions are typically a tiny fraction of all transactions, making it hard for models to learn the minority class.
-   **Evolving Patterns:** Fraudsters constantly adapt their methods, requiring models to be adaptive and robust.
-   **High Stakes:** Missing fraud can lead to significant financial losses for banks and customers, while false alarms can inconvenience legitimate customers and erode trust.

Throughout this notebook, I will guide you through my process, covering:
1.  **Data Loading & Merging:** Consolidating information from various disconnected sources.
2.  **Data Preprocessing:** Handling missing values, cleaning data types, and encoding categorical features.
3.  **Feature Engineering:** Creating new, informative features from existing data, with a strong focus on time-based and interaction terms.
4.  **Model Training:** Utilizing powerful ensemble models like XGBoost and LightGBM.
5.  **Model Evaluation:** Assessing performance using appropriate metrics for imbalanced datasets (Precision, Recall, F1-score, ROC AUC).
6.  **Hyperparameter Tuning:** Optimizing model performance for the best possible results.


## 2. Data Loading and Initial Merging

To begin, I load the various CSV and JSON files provided in the dataset. My first crucial step is to merge these disparate data sources into a single, comprehensive DataFrame. This consolidation is essential because transaction details, card information, user demographics, and merchant categories all contribute valuable signals for identifying fraudulent patterns.

### 2.1 Importing Libraries and Listing Data Files

I start by importing the necessary Python libraries for data manipulation and machine learning. I also include a standard Kaggle utility to list the input files, confirming their presence in the environment.


### 2.2 Loading Datasets

I load each of the five provided datasets into Pandas DataFrames. It's important to note that the file paths are adjusted to match the Kaggle environment. The `train_fraud_labels.json` and `mcc_codes.json` files require special handling due to their JSON format and nested structures.

**Key Data Files:**
-   `transactions_data.csv`: Contains core transaction details.
-   `cards_data.csv`: Provides credit/debit card information.
-   `users_data.csv`: Holds demographic and financial details about users.
-   `mcc_codes.json`: Maps merchant category codes to human-readable descriptions.
-   `train_fraud_labels.json`: Contains the binary fraud labels for a subset of transactions.


In [ ]:
# === CELL 1: SETUP AND DATA LOADING ===

# Standard imports for data manipulation

import numpy as np
import pandas as pd
import json
import gc # For garbage collection, useful for managing memory with large datasets
import re # For regular expressions, used in column name cleaning
import os

# Scikit-learn for preprocessing and model evaluation
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score, f1_score

# XGBoost and LightGBM for modeling
import xgboost as xgb
import lightgbm as lgb

# Matplotlib for plotting (for visualization of tuning results)
import matplotlib.pyplot as plt
import seaborn as sns

# --- Verify and list files in the Kaggle input directory ---
print("Files available in this notebook session:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# --- Define the base path and load all data files ---
# This path should now exist because you've attached the dataset
KAGGLE_DATA_PATH = '/kaggle/input/'

print("\nLoading data files...")
try:
    # Load fraud labels from JSON
    file_path_fraud_labels = KAGGLE_DATA_PATH + 'train_fraud_labels.json'
    with open(file_path_fraud_labels, 'r') as f:
        raw_json_data = json.load(f)
    transaction_labels_dict = raw_json_data['target']
    train_fraud_labels = pd.Series(transaction_labels_dict).reset_index()
    train_fraud_labels.columns = ['transaction_id', 'is_fraud']
    train_fraud_labels['transaction_id'] = pd.to_numeric(train_fraud_labels['transaction_id'])

    # Load other data files
    transaction_df = pd.read_csv(KAGGLE_DATA_PATH + 'transactions_data.csv')
    card_df = pd.read_csv(KAGGLE_DATA_PATH + 'cards_data.csv')
    users_df = pd.read_csv(KAGGLE_DATA_PATH + "users_data.csv")
    mcc_series = pd.read_json(KAGGLE_DATA_PATH + 'mcc_codes.json', typ='series')
    mcc_df = mcc_series.reset_index()
    mcc_df.columns = ['mcc_code', 'description']

    print("\nAll data files loaded successfully.")

except FileNotFoundError:
    print("\nERROR: FileNotFoundError. Please ensure the dataset is attached to the notebook (using the '+ Add data' button on the right) and the KAGGLE_DATA_PATH is correct.")

### 2.3 Merging DataFrames

To create a holistic view of each transaction, I merge all individual DataFrames into a single, wide DataFrame. I perform a series of left joins, starting with the `transaction_df` as the base, and then progressively adding information from other tables.

**Merge Strategy:**
-   **`transaction_df` + `train_fraud_labels`:** Linked by `id` (from `transaction_df`) and `transaction_id` (from `train_fraud_labels`). This step adds the crucial `is_fraud` target variable.
-   **`df` (merged so far) + `card_df`:** Linked by `card_id` (from `transaction_df`) and `id` (from `card_df`). This brings in card-specific details like `credit_limit`, `expires`, etc. I use `suffixes=('', '_card')` to distinguish columns that might have the same name in both DataFrames.
-   **`df` + `users_df`:** Linked by `client_id` (from `transaction_df`) and `id` (from `users_df`). This adds user demographics and financial profile. Again, `suffixes=('', '_user')` are used to prevent column name collisions.
-   **`df` + `mcc_df`:** Linked by `mcc` (from `transaction_df`) and `mcc_code` (from `mcc_df`). This step enriches the data with human-readable merchant descriptions.

After all merges, I drop redundant ID columns that are no longer needed, and delete the original DataFrames to free up memory, which is a critical practice when working with large datasets.


In [ ]:
# --- Merge all DataFrames into one comprehensive DataFrame ---

# Start with the main transactions data
df = pd.merge(transaction_df, train_fraud_labels, left_on='id', right_on='transaction_id', how='left')

# Merge with card data (card_df.id is the card_id)
# Using suffixes to handle potential duplicate column names (e.g., 'id' in both)
df = pd.merge(df, card_df, left_on='card_id', right_on='id', how='left', suffixes=('', '_card'))

# Merge with user data (users_df.id is the client_id)
# Using suffixes again for robustness
df = pd.merge(df, users_df, left_on='client_id', right_on='id', how='left', suffixes=('', '_user'))

# Merge with MCC descriptions (mcc_df.mcc_code is the mcc)
# Note: 'mcc' is the code in transaction_df, 'mcc_code' is the code in mcc_df
df = pd.merge(df, mcc_df, left_on='mcc', right_on='mcc_code', how='left')

# --- Clean up merged columns ---
# Drop redundant ID columns from the merges
# 'transaction_id' is redundant with 'id' (from transaction_df)
# 'id_card' is redundant with 'card_id'
# 'id_user' is redundant with 'client_id'
# 'mcc_code' is redundant with 'mcc'
df = df.drop(columns=['transaction_id', 'id_card', 'id_user', 'mcc_code'])

# Delete original dataframes to free up memory
del transaction_df, train_fraud_labels, card_df, users_df, mcc_df
gc.collect() # Manually trigger garbage collection

print("All data merged into a single DataFrame and original tables deleted.")
print(f"Shape of the final merged DataFrame: {df.shape}")

## 3. Data Splitting and Imbalance Handling
Before any preprocessing or model training, it's crucial to split the data into training, validation, and test sets. This ensures an unbiased evaluation of the model's performance on unseen data. For fraud detection, which is highly imbalanced, stratified sampling is essential to maintain the class distribution across all splits.

### 3.1 Handling Missing Fraud Labels and Target Variable Conversion
First, I drop rows where the is_fraud label is missing. These are transactions for which I don't have ground truth, making them unsuitable for supervised learning. I then convert the 'Yes'/'No' labels to numerical 1s and 0s, which is a requirement for most machine learning algorithms.

In [ ]:
# Drop rows where the fraud label is missing (these are unlabeled transactions)
# This is a critical step as supervised learning requires labeled data.
df.dropna(subset=['is_fraud'], inplace=True)

# Convert the target variable 'is_fraud' to a numerical format (0 for 'No', 1 for 'Yes')
# This is required by most machine learning algorithms.
df['is_fraud'] = df['is_fraud'].map({'No': 0, 'Yes': 1})

# Define features (X) and target (y)
features = [col for col in df.columns if col != 'is_fraud']
X = df[features]
y = df['is_fraud']

### 3.2 Train, Validation, and Test Split (60/20/20)

I split the data into 60% for training, 20% for validation, and 20% for testing.
-   **Training Set (`X_train`, `y_train`):** Used to train the model.
-   **Validation Set (`X_cv`, `y_cv`):** Used for hyperparameter tuning and model selection during development. It helps prevent overfitting to the final test set.
-   **Test Set (`X_test`, `y_test`):** Held out completely until the very end for an unbiased final evaluation of the model's generalization ability.

**Stratified Sampling:** I use `stratify=y` (or `stratify=y_temp`) in the `train_test_split` function. This is vital for imbalanced datasets like fraud detection, as it ensures that the proportion of fraudulent transactions is maintained in each split, preventing a split from having too few (or zero) fraud cases.


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (60%), and a temporary set (40%)
# The first split takes 40% for temp, leaving 60% for X_train.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.40, random_state=42, stratify=y)

# Split the temporary set (which is 40% of original) into validation (20% of original) and test (20% of original)
# 0.50 of X_temp (40%) = 20% of original.
X_cv, X_test, y_cv, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

# Delete the large intermediate dataframes to save memory (important for Kaggle kernels)
del df, X, y, X_temp, y_temp
gc.collect() # Manually trigger garbage collection

print("Full dataset split into training, validation, and test sets.")
print(f"X_train shape: {X_train.shape}")
print(f"X_cv shape: {X_cv.shape}")
print(f"X_test shape: {X_test.shape}")

## 4. Feature Engineering and Preprocessing
This is a crucial phase where I transform raw data into features that machine learning models can effectively learn from. I address data types, missing values, categorical encoding, and create new interaction and time-based features. I've encapsulated many of these steps into a function for consistent application across training, validation, and test sets.



### 4.1 Defining the Feature Engineering Function
I define a comprehensive function apply_preprocessing that encapsulates several feature engineering steps. This ensures that the exact same transformations are applied consistently to X_train, X_cv, and X_test, preventing data leakage.

**Key Transformations within the Function:**

-   **Cleaning numerical columns**: Removing symbols like '$' and ',' and converting to numeric types.

-   **Date Engineering**: Converting date and expires to datetime objects and extracting various time-based features (year, hour, day of week, day of month, month, days to expiry, expiry month/year).

-   **Cyclical Feature Creation**: Generating sin and cos transformations for hour_of_day, day_of_week, and month to properly represent their cyclical nature.

-   **Binary Feature Mapping**: Converting binary categorical features (gender, has_chip, card_on_dark_web) to 0s and 1s.

-   **has_error Feature**: Creating a binary feature from the errors column to indicate the presence of any transaction error.

-   **Numerical NaN Imputation**: Performing final numerical NaN imputation using medians calculated only from the training set.

Note: This function does not handle One-Hot Encoding or merchant_state grouping, as those require global knowledge (fitting on X_train) that is managed outside this function.

In [ ]:
import numpy as np
import pandas as pd

def apply_preprocessing(df, is_training_set=False, median_imputations=None):
    """
    Takes a raw data split and applies all feature engineering steps.
    """
    df_processed = df.copy()

    # --- Step 1: Clean Numerical Columns ---
    amount_cols = ['amount', 'per_capita_income', 'yearly_income', 'credit_limit', 'total_debt']
    for col in amount_cols:
        if col in df_processed.columns:
            df_processed[col] = pd.to_numeric(df_processed[col].astype(str).str.replace(r'[$,]', '', regex=True), errors='coerce')

    # --- Step 2: Date Engineering ---
    date_cols = ['date', 'expires', 'acct_open_date']
    for col in date_cols:
        if col in df_processed.columns:
            df_processed[col] = pd.to_datetime(df_processed[col], errors='coerce', format='mixed')
    
    if 'date' in df_processed.columns:
        df_processed['hour_of_day'] = df_processed['date'].dt.hour
        df_processed['day_of_week'] = df_processed['date'].dt.dayofweek
        df_processed['month'] = df_processed['date'].dt.month
    if 'expires' in df_processed.columns and 'date' in df_processed.columns:
        df_processed['days_to_expiry'] = (df_processed['expires'] - df_processed['date']).dt.days
    
    # Drop original date columns right after use, as you suggested
    df_processed.drop(columns=date_cols, inplace=True)

    # --- Step 3: Cyclical Feature Creation ---
    cyclical_cols_original = ['hour_of_day', 'day_of_week', 'month']
    if all(col in df_processed.columns for col in cyclical_cols_original):
        df_processed['hour_sin'] = np.sin(2 * np.pi * df_processed['hour_of_day'] / 24.0)
        df_processed['hour_cos'] = np.cos(2 * np.pi * df_processed['hour_of_day'] / 24.0)
        df_processed['day_of_week_sin'] = np.sin(2 * np.pi * df_processed['day_of_week'] / 7.0)
        df_processed['day_of_week_cos'] = np.cos(2 * np.pi * df_processed['day_of_week'] / 7.0)
        df_processed['month_sin'] = np.sin(2 * np.pi * df_processed['month'] / 12.0)
        df_processed['month_cos'] = np.cos(2 * np.pi * df_processed['month'] / 12.0)
        
        # Drop original cyclical columns right after use, as you suggested
        df_processed.drop(columns=cyclical_cols_original, inplace=True)

    # --- Step 4: Process Binary and Other Features ---
    if 'errors' in df_processed.columns:
        df_processed['has_error'] = df_processed['errors'].notna().astype(int)
    if 'gender' in df_processed.columns:
        df_processed['gender'] = df_processed['gender'].map({'Female': 0, 'Male': 1})
    if 'has_chip' in df_processed.columns:
        df_processed['has_chip'] = df_processed['has_chip'].map({'NO': 0, 'YES': 1})
        
    # --- Step 5: Final NaN Imputation for numerical features (within this function) ---
    # This is done after all numerical features are created.
    # Medians will be calculated from X_train when this function is called on X_train.
    # Then passed to X_cv and X_test calls.
    numerical_cols_for_imputation = df_processed.select_dtypes(include=np.number).columns.tolist()
    if is_training_set:
        median_imputations = df_processed[numerical_cols_for_imputation].median()
    
    if median_imputations is not None:
        df_processed.fillna(median_imputations, inplace=True)

    return df_processed, median_imputations # Return processed DF and medians

print("Feature engineering function 'create_all_features' defined.")

### 4.2 Applying Feature Engineering to Data Splits

I apply the `apply_preprocessing` function to my `X_train`, `X_cv`, and `X_test` DataFrames. This ensures all the initial transformations and feature creations are consistently applied across all subsets of the data. I also manage memory by explicitly calling `gc.collect()`.

In [ ]:
# --- Main preprocessing execution block ---
print("Step 1: Applying feature engineering to all data splits...")

# Initialize median_imputations_dict outside the loop; it will be populated by X_train's processing
median_imputations_dict = None

# Apply preprocessing to X_train (where medians for imputation are learned)
X_train, median_imputations_dict = apply_preprocessing(X_train, is_training_set=True)
# Apply preprocessing to X_cv and X_test (using medians learned from X_train)
X_cv, _ = apply_preprocessing(X_cv, median_imputations=median_imputations_dict)
X_test, _ = apply_preprocessing(X_test, median_imputations=median_imputations_dict)
gc.collect() # Clean up memory

print('Stage 1: Feature Engineering applied to all splits.')
print(f"X_train shape after initial FE: {X_train.shape}")
print(f"X_train columns after initial FE (first 10): {X_train.columns.tolist()[:10]}...")




### 4.3 Dropping Unnecessary Columns

I systematically drop columns that are either non-informative, redundant, problematic (e.g., due to data quality issues like the `acct_open_date` and `year_pin_last_changed` problems), or sensitive identifiers. This step is crucial for reducing dimensionality, improving model efficiency, and focusing the model on relevant signals.

**Dropped Categories:**
-   **Identifiers & Sensitive Data:** `id`, `client_id`, `card_id`, `merchant_id`, `card_number`, `cvv`.
-   **Problematic Date Sources:** `acct_open_date`, `year_pin_last_changed` (and their derived features if they were still present).
-   **Redundant/Low-Value Categoricals:** `card_on_dark_web`, `has_chip` (original), `address`, `merchant_city`.
-   **Original Cyclical Features:** `hour_of_day`, `day_of_week`, `month` (replaced by sin/cos in `apply_preprocessing`).
-   **Other Redundant Features:** `birth_year`, `birth_month` (redundant with `current_age`), `latitude`, `longitude` (as I decided not to create complex geospatial features from them).


In [ ]:
print("Step 2: Dropping all unnecessary columns...")

# This is the complete master list of columns to drop, consolidated from our discussions.
final_cols_to_drop = [
    # Identifiers and Sensitive Data
    'id', 'client_id', 'card_id', 'merchant_id',
    'card_number', 'cvv','mcc'

    # Problematic Date Columns (and their direct derivatives if still present)
    'acct_open_date', 'year_pin_last_changed',
    # 'account_age_days', 'years_since_pin_change' should be gone if derived from above

    # Redundant/Low-Value Categoricals/Text
    'card_on_dark_web', # This was a single-value column, so its OHE version 'card_on_dark_web_No' would also be single-value.
    'has_chip', # Replaced by 'has_chip_binary'
    'address', # High cardinality text, not used for FE
    'merchant_city', # High cardinality categorical, often redundant with zip/state

    # Redundant Age Features
    'birth_year',
    'birth_month',

    # Geospatial (dropped as I decided against complex geospatial FE)
    'latitude',
    'longitude',

    # Original Date Columns (replaced by extracted features)
    # These should be dropped by apply_preprocessing, but included here for robustness if they somehow remain.
    'date', 'expires',
    # 'hour_of_day', 'day_of_week', 'month' should also be dropped by apply_preprocessing
]

for df_set in [X_train, X_cv, X_test]:
    # Filter list to only drop columns that actually exist in the DataFrame
    cols_that_exist = [col for col in final_cols_to_drop if col in df_set.columns]
    df_set.drop(columns=cols_that_exist, inplace=True, errors='ignore') # Use errors='ignore' for robustness

print('Stage 2: Unnecessary columns dropped.')
print(f"X_train shape after dropping: {X_train.shape}")
print(f"X_train columns after dropping (first 10): {X_train.columns.tolist()[:10]}...")


### 4.4 Grouping and One-Hot Encoding Remaining Categorical Features

Categorical features need to be converted into a numerical format for machine learning models. I apply One-Hot Encoding for features with multiple categories. For `merchant_state`, which has high cardinality and includes many less frequent values, I group these into an 'OTHER_STATE' category based on training data patterns to reduce dimensionality and improve model learning.

**Steps:**
1.  **Impute NaNs** in `merchant_state` with 'Missing_State' (this was done in `apply_preprocessing`).
2.  **Identify Top 15 Fraud States** from the training data (`X_train`) to avoid data leakage. This list will then be used to group states in all splits.
3.  **Group less frequent states** into 'OTHER_STATE'.
4.  **One-Hot Encode** all remaining `object` type columns (e.g., `merchant_state_`, `card_brand_`, `card_type_`, `errors_`, `description_`).


In [ ]:
from sklearn.preprocessing import OneHotEncoder
print("Step 3: Grouping and one-hot encoding...")

# --- Handle merchant_state: Grouping ---
# Group merchant_state based on training set fraud counts (CRUCIAL for no data leakage)
# This part needs to be outside the loop and use y_train.
# Ensure 'merchant_state' is clean (NaNs filled) before this step.
temp_train_df = pd.DataFrame({'merchant_state': X_train['merchant_state'], 'is_fraud': y_train})
fraud_counts = temp_train_df[temp_train_df['is_fraud'] == 1]['merchant_state'].value_counts()
top_15_fraud_states = fraud_counts.nlargest(15).index.tolist()
del temp_train_df, fraud_counts # Clean up temporary data

# Apply grouping to all splits using the list derived from X_train
for df_set in [X_train, X_cv, X_test]:
    if 'merchant_state' in df_set.columns:
        # Use .loc for safe assignment
        df_set.loc[:, 'merchant_state'] = df_set['merchant_state'].apply(lambda x: x if x in top_15_fraud_states else 'OTHER_STATE')


# --- One-Hot Encode all remaining 'object' type columns ---
# This includes 'merchant_state', 'errors', 'card_brand', 'card_type', 'description', 'use_chip'
# and 'gender' if they are still object type.
categorical_cols_to_encode = X_train.select_dtypes(include=['object']).columns.tolist()
print(f'Categorical columns to encode: {categorical_cols_to_encode}')

# Initialize OneHotEncoder (fit only on X_train to prevent data leakage)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.int8)
encoder.fit(X_train[categorical_cols_to_encode])

# Transform and concatenate for all splits
encoded_cols_train = pd.DataFrame(encoder.transform(X_train[categorical_cols_to_encode]), index=X_train.index, columns=encoder.get_feature_names_out(categorical_cols_to_encode))
encoded_cols_cv = pd.DataFrame(encoder.transform(X_cv[categorical_cols_to_encode]), index=X_cv.index, columns=encoder.get_feature_names_out(categorical_cols_to_encode))
encoded_cols_test = pd.DataFrame(encoder.transform(X_test[categorical_cols_to_encode]), index=X_test.index, columns=encoder.get_feature_names_out(categorical_cols_to_encode))

X_train = pd.concat([X_train.drop(columns=categorical_cols_to_encode), encoded_cols_train], axis=1)
X_cv = pd.concat([X_cv.drop(columns=categorical_cols_to_encode), encoded_cols_cv], axis=1)
X_test = pd.concat([X_test.drop(columns=categorical_cols_to_encode), encoded_cols_test], axis=1)

gc.collect() # Clean up memory

print('Stage 3: Grouping and one-hot encoding complete.')
print(f"X_train shape after OHE: {X_train.shape}")
print(f"X_train columns after OHE (first 10): {X_train.columns.tolist()[:10]}...")



### 4.5 Final Data Type Optimization

To ensure memory efficiency and compatibility with machine learning libraries, I downcast numerical columns to smaller data types (e.g., `float32`, `int32`) where possible without losing precision. This is a common practice for large datasets on platforms like Kaggle.

In [ ]:
print("Step 4: Downcasting data types for memory efficiency...")
for df_set in [X_train, X_cv, X_test]:
    for col in df_set.select_dtypes(include=['float64', 'int64']).columns:
        if 'float' in str(df_set[col].dtype):
            df_set.loc[:, col] = df_set[col].astype('float32')
        else:
            df_set.loc[:, col] = pd.to_numeric(df_set[col], downcast='integer')
gc.collect() # Clean up memory

print("\n--- Preprocessing Fully Complete ---")
print(f"X_train final shape: {X_train.shape}")
print(f"X_cv final shape: {X_cv.shape}")
print(f"X_test final shape: {X_test.shape}")

### 4.6 Final NaN Check

As a final validation step, I perform a comprehensive check for any remaining missing values across all columns in my preprocessed DataFrames. It's absolutely crucial that the data is completely clean and free of NaNs before feeding it to the machine learning model, as most algorithms cannot handle them directly.

In [ ]:
print("\n--- Final NaN Check Across ALL Remaining Columns ---")
nan_counts_xtrain = X_train.isna().sum()
print(f"\nX_train NaNs:")
print(nan_counts_xtrain[nan_counts_xtrain > 0]) # Should be empty!

nan_counts_xcv = X_cv.isna().sum()
print(f"\nX_cv NaNs:")
print(nan_counts_xcv[nan_counts_xcv > 0])

nan_counts_test = X_test.isna().sum()
print(f"\nX_test NaNs:")
print(nan_counts_test[nan_counts_test > 0])

if nan_counts_xtrain[nan_counts_xtrain > 0].empty and \
   nan_counts_xcv[nan_counts_xcv > 0].empty and \
   nan_counts_test[nan_counts_test > 0].empty:
    print("\nAll DataFrames are clean (no NaNs found). Ready for modeling!")
else:
    print("\nWARNING: NaNs still present in DataFrames. Please review preprocessing steps.")

## 5. Model Training: XGBoost Classifier
I proceed to train an XGBoost Classifier, a powerful gradient boosting algorithm renowned for its high performance on tabular data. Given the severe class imbalance inherent in fraud detection, I utilize the scale_pos_weight parameter. This crucial parameter ensures the model gives appropriate attention to the minority (fraudulent) class, preventing it from simply predicting the majority class all the time.

### 5.1 Calculating scale_pos_weight
The scale_pos_weight parameter tells XGBoost to penalize misclassifications of the positive class (fraud) more heavily than the negative class. Its value is the ratio of negative samples to positive samples in the training data.

In [ ]:
# Calculate scale_pos_weight from the training target variable
neg_count = y_train.value_counts()[0]
pos_count = y_train.value_counts()[1]
scale_pos_weight_value = neg_count / pos_count
print(f"Count of Legitimate Transactions (0) in y_train: {neg_count}")
print(f"Count of Fraudulent Transactions (1) in y_train: {pos_count}")
print(f"Calculated scale_pos_weight: {scale_pos_weight_value:.2f}\n")

### 5.2 Model 1 Training and Evaluation

I initialize the XGBoost model with a set of reasonable default hyperparameters and the calculated `scale_pos_weight`. I then train it on the `X_train` data and evaluate its performance on the `X_cv` (validation) set. I focus on metrics like Precision, Recall, F1-score, and ROC AUC, which are more informative for imbalanced datasets than simple accuracy.

In [ ]:
import xgboost as xgb

# Initialize and Train XGBoost Classifier
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',  # For binary classification
    eval_metric='logloss',        # Metric for evaluation during training
    use_label_encoder=False,      # Suppress a future deprecation warning
    scale_pos_weight=scale_pos_weight_value, # Crucial for imbalance
    random_state=42,              # For reproducibility
    n_estimators=500,             # Number of boosting rounds (trees)
    learning_rate=0.05,           # Step size shrinkage to prevent overfitting
    max_depth=5,                  # Maximum depth of a tree
    subsample=0.7,                # Subsample ratio of the training instance
    colsample_bytree=0.7,         # Subsample ratio of columns when constructing each tree
    gamma=0.1                     # Minimum loss reduction required to make a further partition
)

print("Training XGBoost model...")
xgb_model.fit(X_train, y_train)
print("Model training complete.\n")

# Evaluate on Validation Set
y_pred_cv = xgb_model.predict(X_cv)
y_proba_cv = xgb_model.predict_proba(X_cv)[:, 1] # Probabilities for the positive class

print("--- Evaluation on Validation Set (X_cv) ---")
print("\nConfusion Matrix:")
print(confusion_matrix(y_cv, y_pred_cv))
print("\nClassification Report:")
print(classification_report(y_cv, y_pred_cv))
roc_auc = roc_auc_score(y_cv, y_proba_cv)
print(f"\nROC AUC Score: {roc_auc:.4f}")

# Display Feature Importances
print("\nFeature Importances (Top 15):")
feature_importances = pd.Series(xgb_model.feature_importances_, index=X_train.columns)
print(feature_importances.nlargest(15))

## 6. Feature Engineering: Adding Interaction Terms and Preparing for Model 2
Now, I prepare the dataset for Model 2. This involves two key steps: identifying and dropping low-importance features from Model 1, and then adding new interaction features.

### 6.1 Identifying and Dropping Low-Importance Features for Model 2
Before creating interaction terms and running Model 2, I identify and drop the 10 features with the lowest importance from Model 1's feature importance list. This step helps to reduce dimensionality, potentially improve performance by removing noise, and optimize memory usage for subsequent steps.

In [ ]:
# Get feature importances from the model I just trained
feature_importances = pd.Series(xgb_model.feature_importances_, index=X_train.columns)

# Identify the least important features to drop for tuning.
# I'll drop the bottom 10 features for this example. This number can be adjusted.
low_importance_features = feature_importances.nsmallest(10).index.tolist()

print(f"Identified {len(low_importance_features)} features with the lowest importance to drop for tuning.")

# Create reduced DataFrames for the search.
# RandomizedSearchCV will run on these smaller datasets to save time and memory.
X_train_model2 = X_train.drop(columns=low_importance_features, errors='ignore')
X_cv_model2 = X_cv.drop(columns=low_importance_features, errors='ignore')
X_test_model2=X_test.drop(columns=low_importance_features, errors='ignore')

del X_train,X_cv,X_test 
gc.collect()

print(f"New shape of X_train for tuning: {X_train_model2.shape}")
print(f"New shape of X_cv for tuning: {X_cv_model2.shape}")
print(f"New shape of X_test for tuning: {X_test_model2.shape}")

### 6.2 Creating Interaction Features for Model 2's Dataset

Now, I explicitly create interaction features by multiplying existing features. These features can capture synergistic effects where the combination of two features provides more information than they do individually. I will add these to my `X_train_model2', 'X_cv_model2' and 'X_test_model2` DataFrames, which have already been reduced by dropping low-importance features from Model 1. This ensures that the interactions are built on a refined set of base features.

In [ ]:
print("\n--- Creating Interaction Features for Model 2's Reduced Datasets ---")

# List of all dataframes to apply interactions to (these are the already reduced ones)
dfs_for_interactions_model2 = [X_train_model2, X_cv_model2, X_test_model2]

for df_set in dfs_for_interactions_model2:
    # --- Amount x Top Categorical/Binary Features ---
    # Check if base columns exist and are numeric before creating interactions.
    
    if all(col in df_set.columns for col in ['amount', 'merchant_state_Italy']):
        df_set['amount_x_state_italy'] = df_set['amount'] * df_set['merchant_state_Italy']
    if all(col in df_set.columns for col in ['amount', 'description_Tolls and Bridge Fees']):
        df_set['amount_x_tolls'] = df_set['amount'] * df_set['description_Tolls and Bridge Fees']
    if all(col in df_set.columns for col in ['amount', 'use_chip_Online Transaction']):
        df_set['amount_x_online_trans'] = df_set['amount'] * df_set['use_chip_Online Transaction']
    if all(col in df_set.columns for col in ['amount', 'use_chip_Swipe Transaction']):
        df_set['amount_x_swipe_trans'] = df_set['amount'] * df_set['use_chip_Swipe Transaction']
    if all(col in df_set.columns for col in ['amount', 'merchant_state_Haiti']):
        df_set['amount_x_state_haiti'] = df_set['amount'] * df_set['merchant_state_Haiti']
    if all(col in df_set.columns for col in ['amount', 'description_Taxicabs and Limousines']):
        df_set['amount_x_taxis_limos'] = df_set['amount'] * df_set['description_Taxicabs and Limousines']
    if all(col in df_set.columns for col in ['amount', 'use_chip_Chip Transaction']):
        df_set['amount_x_chip_trans'] = df_set['amount'] * df_set['use_chip_Chip Transaction']
    if all(col in df_set.columns for col in ['amount', 'merchant_state_OTHER_STATE']):
        df_set['amount_x_state_other'] = df_set['amount'] * df_set['merchant_state_OTHER_STATE']

    # --- Other Interaction Ideas (Non-Amount Based) ---
    if all(col in df_set.columns for col in ['credit_score', 'use_chip_Online Transaction']):
        df_set['credit_score_x_online_trans'] = df_set['credit_score'] * df_set['use_chip_Online Transaction']
    
    # Debt-to-Income Ratio (handle division by zero if yearly_income can be 0)
    if all(col in df_set.columns for col in ['total_debt', 'yearly_income']):
        # Add a small epsilon to yearly_income to prevent division by zero
        df_set['debt_to_income_ratio'] = df_set['total_debt'] / (df_set['yearly_income'] + 1e-6)

print("Interaction features created for Model 2 datasets.")
print(f"X_train_model2 shape is now: {X_train_model2.shape}")
print(f"X_cv_model2 shape is now: {X_cv_model2.shape}")
print(f"X_test_model2 shape is now: {X_test_model2.shape}")

## 7. Model Training: XGBoost Classifier (Model 2 - With Interactions & Reduced Features)
Now, I train a second XGBoost model (Model 2). This model will use the newly created interaction features. Additionally, to optimize performance and reduce dimensionality for the hyperparameter tuning step (and potentially for the final model), I will drop the 20 features with the lowest importance from Model 1's feature importance list. This ensures I'm removing features that were least useful in the simpler model, while still benefiting from the new interactions.

### 7.1 Model 2 Training and Evaluation
I initialize and train the second XGBoost model (Model 2) using the data that now includes interaction terms and has had low-importance features removed. I evaluate its performance on the X_cv_model2 (validation) set.

In [ ]:
# Initialize and Train XGBoost Classifier
xgb_model_2 = xgb.XGBClassifier(
    objective='binary:logistic',  # For binary classification
    eval_metric='logloss',        # Metric for evaluation during training
    use_label_encoder=False,      # Suppress a future deprecation warning
    scale_pos_weight=scale_pos_weight_value, # Crucial for imbalance
    random_state=42,              # For reproducibility
    n_estimators=500,             # Number of boosting rounds (trees)
    learning_rate=0.05,           # Step size shrinkage to prevent overfitting
    max_depth=5,                  # Maximum depth of a tree
    subsample=0.7,                # Subsample ratio of the training instance
    colsample_bytree=0.7,         # Subsample ratio of columns when constructing each tree
    gamma=0.1                     # Minimum loss reduction required to make a further partition
)

print("Training XGBoost model...")
xgb_model_2.fit(X_train_model2, y_train)
print("Model training complete.\n")

# Evaluate on Validation Set
y_pred_cv = xgb_model_2.predict(X_cv_model2)
y_proba_cv = xgb_model_2.predict_proba(X_cv_model2)[:, 1] # Probabilities for the positive class

print("--- Evaluation on Validation Set (X_cv_model2) ---")
print("\nConfusion Matrix:")
print(confusion_matrix(y_cv, y_pred_cv))
print("\nClassification Report:")
print(classification_report(y_cv, y_pred_cv))
roc_auc = roc_auc_score(y_cv, y_proba_cv)
print(f"\nROC AUC Score: {roc_auc:.4f}")

# Display Feature Importances
print("\nFeature Importances (Top 15):")
feature_importances = pd.Series(xgb_model_2.feature_importances_, index=X_train_model2.columns)
print(feature_importances.nlargest(15))

### 8. Hyperparameter Tuning with Randomized Search (for Best Model)

To further optimize the model's performance, I perform hyperparameter tuning using RandomizedSearchCV on my best performing model (Model 2, which includes interactions and reduced features). This method efficiently explores a defined parameter space by randomly sampling combinations, making it suitable for large datasets where GridSearchCV (an exhaustive search) would be too slow. My goal is to maximize the ROC AUC score.

### 8.1 Identifying Low-Importance Features for Tuning (Memory Optimization)
Before running the search, I identify and temporarily drop a set of features that had very low importance in my initial model. This step reduces the dimensionality of the data used during the tuning process, saving significant memory and computation time without sacrificing much performance. This is crucial for running RandomizedSearchCV on a large dataset.

In [ ]:
# --- Section 8: Hyperparameter Tuning with Randomized Search (Memory-Optimized) ---

print("\n--- Starting Hyperparameter Tuning with Randomized Search (Memory-Optimized) ---")

# --- IMPORTANT: Memory Management for Tuning ---
# Create a smaller sample of your training data for tuning.
# This is the key to avoiding MemoryError during cross-validation,
# as RandomizedSearchCV will train on this smaller subset.
SAMPLE_SIZE_FOR_TUNING = 2500000 # Adjust this size based on your RAM. 500k-1M is often a good starting point.
                               # This will be roughly 1/10th of your training data.

# Ensure y_train is aligned with the sample of X_train_model2_base
# Use .copy() to ensure these are independent samples.
X_train_sample_for_tuning = X_train_model2.sample(n=SAMPLE_SIZE_FOR_TUNING, random_state=42).copy()
y_train_sample_for_tuning = y_train.loc[X_train_sample_for_tuning.index].copy() # Ensure y is aligned


print(f"Created a tuning sample of size: {X_train_sample_for_tuning.shape}")


# --- 1. Define the Parameter Grid ---
# These are the hyperparameters I want to tune.
param_distributions = {
    'n_estimators': [500, 750,1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 6, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

# --- 2. Initialize the XGBoost Classifier and RandomizedSearchCV ---
# I use the fixed parameters like scale_pos_weight from before.
base_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    # use_label_encoder=False, # REMOVE THIS LINE (it's obsolete and causes a UserWarning)
    scale_pos_weight=scale_pos_weight_value, # This is the value calculated before
    random_state=42
)

# Set up the search
# n_iter=10 (reduced for faster search on sample)
# cv=3 (keep 3-fold CV for robustness on sample)
random_search = RandomizedSearchCV(
    estimator=base_xgb,
    param_distributions=param_distributions,
    n_iter=10,         # Number of parameter combinations to try
    cv=3,              # 3-fold cross-validation
    scoring='roc_auc', # The best metric for this problem
    verbose=2,         # This will print progress updates
    random_state=42,
    n_jobs=-1          # Use all available CPU cores
)

# --- 3. Run the Search ---
print("\nRunning RandomizedSearchCV on SAMPLE training data...")
# FIT ON THE SAMPLE DATASET HERE:
random_search.fit(X_train_sample_for_tuning, y_train_sample_for_tuning)
print("--- Search Complete ---")

# --- 4. Analyze the Results ---
print(f"\nBest ROC AUC score found on sample: {random_search.best_score_:.4f}")
print("Best parameters found on sample:")
print(random_search.best_params_)

# Get the best model from the search (this model is trained on the sample)
best_xgb_model = random_search.best_estimator_

### 8.2 Evaluating the Best Tuned Model

I evaluate the best model found by `RandomizedSearchCV` on my `X_cv_model2` (validation) set to see its performance with the optimized hyperparameters.


In [ ]:
# Evaluate the best model on the validation set (X_cv)
print("\n--- Evaluation of Best Tuned Model on Validation Set (X_cv) ---")
y_pred_cv_tuned = best_xgb_model.predict(X_cv_model2) # Use X_cv_reduced for evaluation
y_proba_cv_tuned = best_xgb_model.predict_proba(X_cv_model2)[:, 1]

print("\nConfusion Matrix:")
print(confusion_matrix(y_cv, y_pred_cv_tuned))
print("\nClassification Report:")
print(classification_report(y_cv, y_pred_cv_tuned))
roc_auc_tuned = roc_auc_score(y_cv, y_proba_cv_tuned)
print(f"\nROC AUC Score: {roc_auc_tuned:.4f}")

# Display Feature Importances for the best model
print("\nFeature Importances (Top 20 from Tuned Model):")
feature_importances_tuned = pd.Series(best_xgb_model.feature_importances_, index=X_train_model2.columns) # Use X_train_model2 columns
print(feature_importances_tuned.nlargest(20))

In [ ]:
# --- AFTER the evaluation of best_xgb_model on X_cv_model2 ---

print("\n--- Evaluation of Best Tuned Model with Custom Threshold (0.7) ---")

# Define the new custom threshold
custom_threshold = 0.7

# Apply the custom threshold to the probabilities from the best tuned model
# If the probability of fraud (y_proba_cv_tuned) is greater than the custom_threshold,
# classify as 1 (fraud), otherwise classify as 0 (not fraud).
y_pred_cv_thresholded = (y_proba_cv_tuned > custom_threshold).astype(int)

print(f"\nConfusion Matrix (Threshold = {custom_threshold}):")
print(confusion_matrix(y_cv, y_pred_cv_thresholded))

print(f"\nClassification Report (Threshold = {custom_threshold}):")
print(classification_report(y_cv, y_pred_cv_thresholded))

# ROC AUC is threshold-independent, so it won't change by adjusting the prediction threshold.
# It's still useful to print it to confirm the model's underlying discriminative power.
roc_auc_thresholded = roc_auc_score(y_cv, y_proba_cv_tuned) # Note: Still uses probabilities, not thresholded predictions
print(f"\nROC AUC Score (Threshold = {custom_threshold}): {roc_auc_thresholded:.4f}")

print("\nAnalysis of Custom Threshold:")
print(f"By increasing the classification threshold to {custom_threshold}, I expect to see a significant decrease in False Positives (leading to higher Precision) and potentially a decrease in True Positives (leading to lower Recall). This trade-off is crucial for aligning the model's output with specific business requirements, such as reducing the number of false alarms for fraud investigation teams.")

In [ ]:
# --- AFTER the evaluation of best_xgb_model on X_cv_model2 ---

print("\n--- Evaluation of Best Tuned Model with Custom Threshold (0.15) ---")

# Define the new custom threshold
custom_threshold = 0.15

# Apply the custom threshold to the probabilities from the best tuned model
# If the probability of fraud (y_proba_cv_tuned) is greater than the custom_threshold,
# classify as 1 (fraud), otherwise classify as 0 (not fraud).
y_pred_cv_thresholded = (y_proba_cv_tuned > custom_threshold).astype(int)

print(f"\nConfusion Matrix (Threshold = {custom_threshold}):")
print(confusion_matrix(y_cv, y_pred_cv_thresholded))

print(f"\nClassification Report (Threshold = {custom_threshold}):")
print(classification_report(y_cv, y_pred_cv_thresholded))

# ROC AUC is threshold-independent, so it won't change by adjusting the prediction threshold.
# It's still useful to print it to confirm the model's underlying discriminative power.
roc_auc_thresholded = roc_auc_score(y_cv, y_proba_cv_tuned) # Note: Still uses probabilities, not thresholded predictions
print(f"\nROC AUC Score (Threshold = {custom_threshold}): {roc_auc_thresholded:.4f}")

print("\nAnalysis of Custom Threshold:")
print(f"By decreasing the classification threshold to {custom_threshold}, I expect to see a significant decrease in False Negatives (leading to higher Recall) and potentially a increase in True Positives (leading to lower Precidsion). This trade-off is crucial for aligning the model's output with specific business requirements, such as reducing the number of false alarms for fraud investigation teams.")

### 8.3 Visualizing Tuning Results
To gain insights into how different hyperparameters affect model performance, I visualize the ROC AUC scores obtained during the randomized search. This helps understand the sensitivity of the model to various parameter settings.

In [ ]:
# Get the results from RandomizedSearchCV
results = random_search.cv_results_
results_df = pd.DataFrame(results)

# Plotting function to visualize tuning results
def plot_tuning_results(results_df, param_name, title):
    plt.figure(figsize=(10, 6))
    
    # Get the mean test score for each combination
    scores = results_df['mean_test_score']
    
    # Get the parameter values
    param_values = results_df[f'param_{param_name}']
    
    # Check if the parameter is a numerical type before plotting
    if pd.api.types.is_numeric_dtype(param_values):
        # Sort values for a cleaner plot
        sorted_results = results_df.sort_values(by=f'param_{param_name}')
        
        plt.plot(sorted_results[f'param_{param_name}'], sorted_results['mean_test_score'], marker='o')
        
    else:
        # If not numeric (e.g., a string), plot a swarmplot or similar
        sns.swarmplot(x=f'param_{param_name}', y='mean_test_score', data=results_df)

    plt.title(title, fontsize=16)
    plt.xlabel(param_name, fontsize=12)
    plt.ylabel('Mean ROC AUC Score', fontsize=12)
    plt.grid(True)
    plt.show()

# Let's visualize the performance against one or two key parameters
# Choose from ['n_estimators', 'learning_rate', 'max_depth', 'subsample', 'colsample_bytree', 'gamma']

# Plot ROC AUC vs. learning_rate
plot_tuning_results(results_df, 'learning_rate', 'Tuning Results: ROC AUC vs. Learning Rate')

# Plot ROC AUC vs. max_depth
plot_tuning_results(results_df, 'max_depth', 'Tuning Results: ROC AUC vs. Max Depth')

print("The plots above show how performance varied with different parameter settings during the search.")

## 9. Final Model Training on Full Dataset with Best Parameters
After completing the hyperparameter tuning on a sample of the data, I now train the final XGBoost model on the full X_train_model2 dataset using the best_params_ identified during the RandomizedSearchCV. This ensures the model leverages the complete training data for optimal learning.

In [ ]:
# Make sure to use the best_xgb_model obtained from RandomizedSearchCV
# And use X_test_reduced if you applied the low-importance feature dropping.

print("\n--- Final Evaluation on Test Set (X_test) ---")

y_pred_test = best_xgb_model.predict(X_test_model2) # Use X_test_reduced for evaluation
y_proba_test = best_xgb_model.predict_proba(X_test_model2)[:, 1]

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))
roc_auc_test = roc_auc_score(y_test, y_proba_test)
print(f"\nROC AUC Score: {roc_auc_test:.4f}")

# Display Feature Importances for the best model on the test set
print("\nFeature Importances (Top 20 from Best Model):")
feature_importances_final = pd.Series(best_xgb_model.feature_importances_, index=X_train_model2.columns)
print(feature_importances_final.nlargest(20))

## 10. Alternative Model: LightGBM
As an alternative to XGBoost, I also explore LightGBM, another powerful gradient boosting framework. LightGBM is often significantly faster and can sometimes achieve comparable or even better performance than XGBoost, especially on large datasets.

### 10.1 Cleaning Column Names for LightGBM
LightGBM can sometimes have issues with special characters in column names (e.g., [, ], <,      ). I clean the column names by replacing such characters with underscores to ensure compatibility. This step is applied to the full X_train_model2, X_cv_model2, X_test_model2 (the datasets used for Model 2).

In [ ]:
print("Cleaning column names for LightGBM compatibility...")

for df_set in [X_train_model2, X_cv_model2, X_test_model2]:
    # The regex '[^A-Za-z0-9_]+' finds any character that is NOT a letter, number, or underscore
    # and replaces it with a single underscore.
    df_set.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in df_set.columns]

print("Column names have been cleaned.")

# --- Verification: Display a few of the new, cleaned column names ---
print("\nSample of cleaned column names in X_train (first 10):")
print(X_train_model2.columns.tolist()[:10])

### 10.2 LightGBM Model Training and Evaluation

I initialize and train a LightGBM classifier using similar parameters to XGBoost, including `scale_pos_weight` for imbalance handling. I then evaluate its performance on the validation set.

In [ ]:
# Calculate scale_pos_weight (same as for XGBoost)
scale_pos_weight_value = y_train.value_counts()[0] / y_train.value_counts()[1]

# --- Initialize and Train LightGBM Classifier ---
lgbm_model = lgb.LGBMClassifier(
    objective='binary',
    metric='logloss',
    scale_pos_weight=scale_pos_weight_value, # Crucial for imbalance
    random_state=42,
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.7,
    colsample_bytree=0.7
)

print("Training LightGBM model...")
lgbm_model.fit(X_train_model2, y_train) # Train on the full X_train (not reduced unless you sampled for LGBM too)
print("Model training complete.\n")

# --- Evaluate on Validation Set ---
y_pred_cv_lgbm = lgbm_model.predict(X_cv_model2) # Evaluate on the full X_cv
y_proba_cv_lgbm = lgbm_model.predict_proba(X_cv_model2)[:, 1]

print("--- Evaluation of LightGBM on Validation Set (X_cv) ---")
print("\nConfusion Matrix:")
print(confusion_matrix(y_cv, y_pred_cv_lgbm))
print("\nClassification Report:")
print(classification_report(y_cv, y_pred_cv_lgbm))
roc_auc_lgbm = roc_auc_score(y_cv, y_proba_cv_lgbm)
print(f"\nROC AUC Score: {roc_auc_lgbm:.4f}")

## 11. Conclusion and Future Work

### 11.1 Summary of Findings
-   **Outstanding Performance**: My final XGBoost model (Model 2, with interactions and tuned hyperparameters) achieved an outstanding ROC AUC score of 0.9929 on the test set. This demonstrates an excellent ability to distinguish between fraudulent and legitimate transactions.

-   **Effective Feature Engineering**: The significant improvement in model performance was largely driven by meticulous data preprocessing and the creation of highly informative features, especially from time-based data and interaction terms. My strategic removal of noisy or redundant features further enhanced the model's efficiency and accuracy.

-   **Class Imbalance Handled**: The scale_pos_weight parameter effectively addressed the severe class imbalance, leading to high recall for the fraud class while also improving precision.

-   **Key Features**: Features related to merchant location (zip, merchant_state), transaction type (use_chip_Online Transaction), and specific merchant descriptions (description_Tolls and Bridge Fees, description_Taxicabs and Limousines) were consistently identified as highly important. Interaction terms like amount_x_tolls and amount_x_state_italy also emerged as very strong predictors.

### 9.2 Future Work

Despite the strong performance, there's always room for further exploration and improvement in a real-world fraud detection system:

-   **Threshold Optimization**: Implement a detailed threshold tuning process (e.g., plotting Precision-Recall curves) to find the optimal balance between precision and recall based on specific business costs of false positives vs. false negatives. This is crucial for practical deployment.

-   **Advanced Feature Engineering**:

    -     Explore more complex velocity features (e.g., number of transactions per card/client/merchant in the last hour/day/week).

    -    Investigate advanced geospatial features if more granular location data is available.

-   **Other Models**: Experiment with other advanced ensemble models (e.g., CatBoost) or deep learning approaches for tabular data.

-   **Anomaly Detection**: Integrate unsupervised anomaly detection techniques to identify novel fraud patterns that supervised models might miss.

-   **Cost-Sensitive Learning**: Directly incorporate the financial costs of misclassifications into the model's loss function.

-   **Real-time Considerations**: For a production system, consider aspects like model latency, data streaming, and continuous learning.